# 📊 Код ранжирования изображений по тексту с Florence-2

* принимает **папку `/content/images`** с сохранёнными PNG/JPG,

* читает **`prompts.csv`** или `prompts.json` со столбцами/ключами
  `image, prompt, prompt2, negative, negative2`,

* вычисляет для каждого изображения-строки четыре оценки:

| модель                  | что считает                                  | почему                             |
|------------------------|----------------------------------------------|------------------------------------|
| **SigLIP 2 (ViT-L/14)** | глобальное cos( I, T )                       | надёжная мультиязычная «семантика» |
| **Florence-2**          | число фраз из prompt, найденных на картинке | уточнённое локальное выравнивание  |
| **CLIP-IQA (ViT-B/16)** | вероятность «качественного» фото             | штраф за артефакты                 |
| **DINOv2 (ViT-L/14)**   | визуальный «aesthetic score» (ℓ₂-норма CLS) | доп. сигнал «детальности/качества» |

* собирает ансамбль:
  `S = α · SigLIP – α · SigLIP_neg + β · Florence – β · Florence_neg + γ · IQA + δ · DINO`,

* выдаёт `ranking.csv` с итоговым порядком изображений.

| ячейка | что делает                                                                                 |
|--------|--------------------------------------------------------------------------------------------|
| **0**  | установка зависимостей: PyTorch, Transformers, CLIP-IQA, Florence-2 и др.                   |
| **1**  | загрузка моделей: SigLIP2, Florence-2, DINOv2, CLIP-IQA                                    |
| **2**  | обёртки-функции для получения индивидуальных скорингов                                     |
| **3**  | функция `rank_folder`: ранжирует изображения по агрегированному скору                      |
| **4**  | запуск с указанием папки и prompt-файла, сохранение результата в `ranking.csv`             |

> **Совет по весам:**
> Для стабильности нормализуйте оценки (`z-score` по каждому столбцу) — так проще подбирать α, β, γ, δ, t.

---

### 🧠 Что считают модели:

* **SigLIP 2** — глобальная семантика между изображением и всей подсказкой;
* **Florence-2** — phrase grounding: доля слов/фраз prompt, для которых найдены регионы на изображении.;
* **CLIP-IQA** — вероятность того, что изображение качественное (vs «bad photo»);
* **DINOv2** — L2-норма CLS-вектора, коррелирующая с визуальной выразительностью.
* **Blip-2** —
---

Загрузите картинки в `/content/images` и подготовьте `prompts.csv` вида:

```csv
image,prompt,prompt2,negative,negative2
0001.png,"A serene landscape with mountains","A calm river under pink sunset","low quality","distortions"
0002.png,"A serene landscape with mountains","","low quality",""
```

После выполнения всех ячеек получите `ranking.csv` с отсортированными изображениями.
"""

In [1]:
!git clone https://github.com/Mike030668/rank_images_project.git
%cd rank_images_project

Cloning into 'rank_images_project'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 96 (delta 37), reused 92 (delta 33), pack-reused 0 (from 0)
Receiving objects: 100% (96/96), 2.86 MiB | 6.75 MiB/s, done.
Resolving deltas: 100% (37/37), done.
/content/rank_images_project


In [2]:
!pip install --upgrade pip setuptools wheel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 77.2 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [2]:
%cd rank_images_project
!pip install --pre \
  --extra-index-url https://download.pytorch.org/whl/nightly/cu121 \
  -e .

/content/rank_images_project
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cu121
Obtaining file:///content/rank_images_project
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 162.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 160.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 134.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 177.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 208.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 65.7

# Работа с веткой

In [2]:
%cd rank_images_project
# информацию о всех удалённых ветках
!git fetch origin

/content/rank_images_project


In [3]:
# увидеть и локальные, и удалённые:
!git branch -a

* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/feature/blip2-metric
  remotes/origin/feature/workflow_new_metrics
  remotes/origin/main


In [5]:
# Создать новую локальную ветку с именем feature/blip2-metric,
# которая будет отслеживать (track) удалённую ветку origin/feature/blip2-metric
!git checkout feature/blip2-metric


Already on 'feature/blip2-metric'
Your branch is up to date with 'origin/feature/blip2-metric'.


In [7]:
#%cd rank_images_project
!ls -R

.:
data  notebooks  pyproject.toml  README.md  src

./data:
demo_images

./data/demo_images:
1010.png  222.png  444.png  666.png  888.png  prompts.json
111.png   333.png  555.png  777.png  999.png  ranking.csv

./notebooks:
Colab_range_images_Flux.ipynb

./src:
rank_images

./src/rank_images:
cli.py	   data_processing.py  example_metric.py  metrics.py  ranking.py
config.py  device_utils.py     __init__.py	  models.py   utils.py


После успешной установки вы можете протестировать работу CLI

In [1]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [2]:
import logging
import sys

# Настройка базового логирования ДО импортов из rank_images
logging.basicConfig(
    level=logging.DEBUG, # Или logging.INFO
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)] # Вывод в ячейку
)
print("Логирование настроено.")

Логирование настроено.


In [3]:
# https://huggingface.co/docs/transformers/main/en/model_doc/blip-2#transformers.Blip2ForImageTextRetrieval

# Ячейка 1: Настройка и импорт
import sys
from pathlib import Path
from PIL import Image
import logging

# Настройка логирования для отладки
logging.basicConfig(level=logging.DEBUG) # Или logging.INFO для менее подробного вывода

# Убедитесь, что путь к вашему проекту корректен
project_root = Path('/content/rank_images_project') # ИЛИ путь на вашей локальной машине
sys.path.insert(0, str(project_root / "src"))

print("Путь к проекту добавлен в sys.path")
# В тестовой ячейке, перед проверкой
from rank_images import models
print(f"[TEST] models.blip2_model is: {models.blip2_model}")
print(f"[TEST] models.blip2_processor is: {models.blip2_processor}")
if models.blip2_model is not None and models.blip2_processor is not None:
     print("BLIP-2 модель и процессор успешно загружены.")
else:
     print("BLIP-2 модель или процессор НЕ БЫЛИ загружены. Проверьте логи выше.")

# Ячейка 2: Загрузка модели
from rank_images.models import load_models, blip2_model, blip2_processor

print("Начинаю загрузку моделей...")
try:
    load_models()
    print("Загрузка моделей завершена.")

    if blip2_model is not None and blip2_processor is not None:
        print("BLIP-2 модель и процессор успешно загружены.")
        print(f"  Модель на устройстве: {blip2_model.device}")
        print(f"  Тип данных модели: {next(blip2_model.parameters()).dtype}")
    else:
        print("BLIP-2 модель или процессор НЕ БЫЛИ загружены. Проверьте логи выше.")
        # raise RuntimeError("BLIP-2 не загружена")

except Exception as e:
    print(f"Ошибка при загрузке моделей: {e}")
    raise

# Ячейка 3: Тестирование функции метрики
from rank_images.metrics import get_blip2_match_score
import torch

# --- 1. Тест с искусственными данными ---
print("\n--- Тест 1: Простые искусственные данные ---")
try:
    # Создаем очень простое тестовое изображение
    test_img = Image.new('RGB', (224, 224), color = 'red')
    test_prompts = ["a red square", "a blue circle"]

    print(f"Тестовое изображение: {test_img}")
    print(f"Тестовые промпты: {test_prompts}")

    score = get_blip2_match_score(test_img, test_prompts)
    print(f"BLIP-2 Score (искусственные данные): {score}")

except Exception as e:
    print(f"Ошибка в Тесте 1: {e}")
    import traceback
    traceback.print_exc()

# --- 2. Тест с реальным изображением из demo_images ---
print("\n--- Тест 2: Реальное изображение из demo_images ---")
try:
    # Укажите путь к одному из ваших демонстрационных изображений
    demo_img_path = project_root / "data" / "demo_images" / "111.png" # Замените на имя вашего файла

    if not demo_img_path.exists():
        # Попробуем другое стандартное имя
        demo_img_path = project_root / "data" / "demo_images" / "image1.jpg"

    if demo_img_path.exists():
        real_img = Image.open(demo_img_path).convert('RGB')
        # Используем промпт из вашего prompts.json или придумаем подходящий
        real_prompts = ["an image", "a picture"] # Простые, универсальные промпты

        print(f"Реальное изображение: {demo_img_path}")
        print(f"Реальные промпты: {real_prompts}")

        score_real = get_blip2_match_score(real_img, real_prompts)
        print(f"BLIP-2 Score (реальное изображение): {score_real}")
    else:
        print("Файл демонстрационного изображения не найден. Пропуск теста 2.")

except Exception as e:
    print(f"Ошибка в Тесте 2: {e}")
    import traceback
    traceback.print_exc()

# --- 3. Тест на CPU (если GPU вызывает проблемы) ---
print("\n--- Тест 3: Принудительное использование CPU ---")
try:
    # Импортируем напрямую для теста
    from rank_images import models
    import torch

    if models.blip2_model is not None and models.blip2_processor is not None:
        print("Принудительный тест BLIP-2 на CPU...")

        # Создаем простые данные
        dummy_img = Image.new('RGB', (224, 224), color = 'green')
        dummy_prompts = ["a green pixel"]

        # Подготовка данных
        inputs = models.blip2_processor(images=dummy_img, text=dummy_prompts, return_tensors="pt", padding=True)
        print(f"Входные тензоры созданы. Ключи: {list(inputs.keys())}")
        for k, v in inputs.items():
            print(f"  {k}: shape={v.shape}, dtype={v.dtype}")

        # Перемещение на CPU
        model_cpu = models.blip2_model.to(torch.device("cpu"))
        inputs_cpu = {k: v.to(torch.device("cpu")) for k, v in inputs.items()}

        print("Модель и данные перемещены на CPU.")

        # Инференс
        with torch.no_grad(): # Важно для инференса
            outputs = model_cpu(**inputs_cpu)

        print("Инференс на CPU выполнен успешно.")

        # Постобработка
        logits_per_text = outputs.logits_per_text
        probs = torch.nn.functional.softmax(logits_per_text, dim=-1)
        match_probs = probs[:, 1]
        avg_prob = match_probs.mean().item()

        print(f"Score на CPU: {avg_prob}")

        # Возврат модели на исходное устройство (если оно было GPU)
        # models.blip2_model.to(models.blip2_model.device) # Этот шаг может быть не нужен или вызвать ошибку, если модель была на CPU

    else:
        print("BLIP-2 модель не загружена, тест на CPU пропущен.")

except Exception as e:
    print(f"Ошибка в Тесте 3 (CPU): {e}")
    import traceback
    traceback.print_exc()

print("\n--- Тестирование BLIP-2 завершено ---")

Путь к проекту добавлен в sys.path
[TEST] models.blip2_model is: None
[TEST] models.blip2_processor is: None
BLIP-2 модель или процессор НЕ БЫЛИ загружены. Проверьте логи выше.
Начинаю загрузку моделей...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


[DEBUG] BLIP-2 Processor type: <class 'transformers.models.blip_2.processing_blip_2.Blip2Processor'>
[DEBUG] BLIP-2 (ITM) (Model type: <class 'transformers.models.blip_2.modeling_blip_2.Blip2ForImageTextRetrieval'>
Загрузка моделей завершена.
BLIP-2 модель или процессор НЕ БЫЛИ загружены. Проверьте логи выше.

--- Тест 1: Простые искусственные данные ---
Тестовое изображение: <PIL.Image.Image image mode=RGB size=224x224 at 0x7EF3FE2271D0>
Тестовые промпты: ['a red square', 'a blue circle']


Expanding inputs for image tokens in BLIP-2 should be done in processing. Please follow instruction here (https://gist.github.com/zucchini-nlp/e9f20b054fa322f84ac9311d9ab67042) to update your BLIP-2 model. Using processors without these attributes in the config is deprecated and will throw an error in v4.50.


BLIP-2 Score (искусственные данные): 0.46637725830078125

--- Тест 2: Реальное изображение из demo_images ---
Реальное изображение: /content/rank_images_project/data/demo_images/111.png
Реальные промпты: ['an image', 'a picture']
BLIP-2 Score (реальное изображение): 9.73045825958252e-05

--- Тест 3: Принудительное использование CPU ---
Принудительный тест BLIP-2 на CPU...
Входные тензоры созданы. Ключи: ['input_ids', 'attention_mask', 'pixel_values']
  input_ids: shape=torch.Size([1, 5]), dtype=torch.int64
  attention_mask: shape=torch.Size([1, 5]), dtype=torch.int64
  pixel_values: shape=torch.Size([1, 3, 224, 224]), dtype=torch.float32
Модель и данные перемещены на CPU.
Инференс на CPU выполнен успешно.
Ошибка в Тесте 3 (CPU): index 1 is out of bounds for dimension 1 with size 1

--- Тестирование BLIP-2 завершено ---


Traceback (most recent call last):
  File "/tmp/ipython-input-3-659254532.py", line 131, in <cell line: 0>
    match_probs = probs[:, 1]
                  ~~~~~^^^^^^
IndexError: index 1 is out of bounds for dimension 1 with size 1


In [8]:
!rank-images --help

2025-07-24 13:27:23,057 [INFO] numexpr.utils: NumExpr defaulting to 12 threads.
2025-07-24 13:27:30.199506: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-24 13:27:30.216329: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753363650.237885    3829 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753363650.244348    3829 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-24 13:27:30.265728: I tensorflow/core/platform/cpu_feat

In [4]:
!rank-images --demo

2025-07-24 16:06:20,966 [INFO] numexpr.utils: NumExpr defaulting to 12 threads.
2025-07-24 16:06:26.741633: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753373186.762974   46849 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753373186.769385   46849 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-24 16:06:29,933 [INFO] rank_images.cli: Запуск в демонстрационном режиме.
2025-07-24 16:06:29,933 [INFO] rank_images.cli: Папка с изображениями: /content/rank_images_project/data/demo_images
2025-07-24 16:06:29,933 [INFO] rank_images.cli: Файл с промптами: /content/rank_images_project/data/demo_images/prompts.json
2025-07-24 16:06:29,933 [INFO] ra

In [ ]:
!rank-images data/demo_images --prompts data/demo_images/prompts.json

# настройка логирования

In [4]:
# Первая ячейка в Jupyter-ноутбуке или начало cli.py
import logging
import sys

# --- Настройка логирования ---
# 1. Очищаем существующие обработчики (на случай, если basicConfig уже вызывался)
logging.getLogger().handlers.clear()
logging.getLogger().setLevel(logging.NOTSET)

# 2. Создаем новый обработчик, который будет писать в stdout (вывод ячейки)
handler = logging.StreamHandler(sys.stdout)
# 3. Устанавливаем формат сообщений
formatter = logging.Formatter("%(asctime)s [%(levelname)s] %(name)s: %(message)s")
handler.setFormatter(formatter)
# 4. Добавляем обработчик к корневому логгеру
logging.getLogger().addHandler(handler)
# 5. Устанавливаем УРОВЕНЬ логгирования на INFO
#    Это ключевой момент: только сообщения уровня INFO и выше будут отображаться
logging.getLogger().setLevel(logging.INFO)
# --- Конец настройки логирования ---

In [6]:
%cd rank_images_project
import sys
from pathlib import Path
# Предполагаем, что текущая директория - корень проекта
project_root = Path.cwd()
sys.path.insert(0, str(project_root / "src")) # Добавляем src в путь импорта

from rank_images.models import load_models
from rank_images.ranking import rank_folder

# Загружаем модели один раз
print("Загружаю модели...")
load_models()
print("Модели загружены.")


/content/rank_images_project
Загружаю модели...
[DEBUG] BLIP-2 Processor type: <class 'transformers.models.blip_2.processing_blip_2.Blip2Processor'>
[DEBUG] BLIP-2 (ITM) (Model type: <class 'transformers.models.blip_2.modeling_blip_2.Blip2ForImageTextRetrieval'>
Модели загружены.


In [7]:
# Выполняем ранжирование
demo_images_dir = project_root / "data" / "demo_images"
prompts_path = str(demo_images_dir / "prompts.json") # <-- str

print("Начинаю ранжирование...")
result_df = rank_folder(
    img_dir=demo_images_dir,
    #prompts_in="An advertising image for a credit card, featuring prominent 3D rendered numbers representing a high deposit percentage. The scene is set against a stunning natural backdrop of a serene blue ocean meeting majestic, sun-drenched mountains. The overall style should be a vibrant and dynamic 3D render, capturing the feeling of opportunity and natural beauty."
    prompts_in= prompts_path # <-- str
)
print("Ранжирование завершено.")
print(result_df)#.head())

Начинаю ранжирование...


Обработка изображений:   0%|          | 0/10 [00:00<?, ?it/s]

Ранжирование завершено.
      image       sig      flor       iqa      dino     blip2  sig_norm  \
0   777.png  0.207520 -0.324959  0.799303  46.81250  0.999512  1.232570   
1   222.png  0.201965 -0.287735  0.847685  47.06250  0.998291  0.657665   
2  1010.png  0.196503 -0.324959  0.871476  47.21875  0.999756  0.092237   
3   666.png  0.209290 -0.324959  0.666524  46.15625  0.998535  1.415781   
4   888.png  0.194519 -0.361622  0.943873  47.62500  0.999023 -0.113086   
5   999.png  0.190033 -0.324959  0.915174  46.62500  0.999512 -0.577431   
6   333.png  0.187195 -0.295547  0.918014  46.21875  0.999268 -0.871201   
7   111.png  0.182434 -0.324959  0.912280  46.53125  0.999756 -1.363976   
8   444.png  0.205078 -0.436070  0.731113  46.31250  0.999023  0.979864   
9   555.png  0.181580 -0.324959  0.834715  46.78125  0.995605 -1.452423   

   flor_norm  iqa_norm  dino_norm  blip2_norm     total  
0   0.207553 -0.525943   0.175791    0.584349  0.568911  
1   1.159765  0.043162   0.738321 